# Imports and Setup

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
# Added comma for thousands separator, keeping 2 decimals for currency
pd.set_option('display.float_format', '{:,.2f}'.format)

print("Libraries loaded successfully")

Libraries loaded successfully


# Load All Tables

In [11]:
tables = {}
files = [
    'transaction_data', 'hh_demographic', 'product',
    'campaign_desc', 'campaign_table', 'coupon',
    'coupon_redempt', 'causal_data'
]

for file in files:
    path = f'../data/raw/{file}.csv'
    tables[file] = pd.read_csv(path)
    print(f"{file}: {len(tables[file]):,} rows, {len(tables[file].columns)} columns")

trans = tables['transaction_data']

id_cols = ['household_key', 'BASKET_ID', 'PRODUCT_ID', 'STORE_ID', 'WEEK_NO', 'DAY']
for col in id_cols:
    trans[col] = trans[col].astype('int64')

print("Data types corrected: IDs converted to integers.")

transaction_data: 2,595,732 rows, 12 columns
hh_demographic: 801 rows, 8 columns
product: 92,353 rows, 7 columns
campaign_desc: 30 rows, 4 columns
campaign_table: 7,208 rows, 3 columns
coupon: 124,548 rows, 3 columns
coupon_redempt: 2,318 rows, 4 columns
causal_data: 36,786,524 rows, 5 columns
Data types corrected: IDs converted to integers.


# Profile Transaction Data

In [23]:
# Cell 3: Profile Transaction Data & Analyze Outliers (FINAL ALIGNMENT)
# 1. RESET & TYPE ENFORCEMENT
trans = tables['transaction_data'].copy()
prod = tables['product'].copy()

# Force IDs to match exactly (Integer to Integer)
trans['PRODUCT_ID'] = trans['PRODUCT_ID'].astype('int64')
prod['PRODUCT_ID'] = prod['PRODUCT_ID'].astype('int64')

# 2. ENRICH: Merge to get descriptions
print("Merging product descriptions...")
df_analysis = trans.merge(prod[['PRODUCT_ID', 'COMMODITY_DESC']], on='PRODUCT_ID', how='left')
df_analysis['COMMODITY_DESC'] = df_analysis['COMMODITY_DESC'].fillna('UNKNOWN')

# 3. DEFINE LOGIC (Matches clean_transactions.py)

# A. Name Filter (Fuel)
is_fuel_name = df_analysis['COMMODITY_DESC'].str.contains('FUEL|GASOLINE', na=False, case=False)

# B. Economic Filter (Row-Level)
# Identifies administrative items (Points/Tokens) via implied unit price.
unit_price = df_analysis['SALES_VALUE'] / df_analysis['QUANTITY']
is_non_merch = (unit_price < 0.05) & (df_analysis['SALES_VALUE'] > 0)

# Combine Filters (The "Kill List")
outlier_mask = is_fuel_name | is_non_merch

# Create Clean DataFrame
df_clean = df_analysis[~outlier_mask].copy()

# 4. VERIFY RESULTS
print(f"\nOriginal Rows: {len(df_analysis):,}")
print(f"Cleaned Rows:  {len(df_clean):,} (Removed {outlier_mask.sum():,} rows of Fuel/Points)")

print(f"\nOriginal Max Quantity: {df_analysis['QUANTITY'].max():,}")
print(f"Cleaned Max Quantity:  {df_clean['QUANTITY'].max():,}")

print("\n=== TOP 5 CLEANED QUANTITIES (Validation) ===")
# This should show real high-volume items (like Yogurt/Soup), not 23k Points
display(df_clean.nlargest(5, 'QUANTITY')[['PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'COMMODITY_DESC']])

print("\n=== METRIC SUMMARY (Final) ===")
metric_cols = ['QUANTITY', 'SALES_VALUE', 'RETAIL_DISC', 'COUPON_DISC', 'TRANS_TIME']
summary_df = df_clean[metric_cols].describe().drop('count')
display(summary_df.style.format('{:,.2f}'))

Merging product descriptions...

Original Rows: 2,595,732
Cleaned Rows:  2,570,687 (Removed 25,045 rows of Fuel/Points)

Original Max Quantity: 89,638
Cleaned Max Quantity:  144

=== TOP 5 CLEANED QUANTITIES (Validation) ===


,PRODUCT_ID,QUANTITY,SALES_VALUE,COMMODITY_DESC
1541674,911878,144,24.00,CORN
1840749,1014675,90,9.00,STATIONERY & SCHOOL SUPPLIES
528062,879948,74,43.01,CANDY - CHECKLANE
621401,1080279,72,38.95,CANDY - CHECKLANE
1018143,907994,72,18.00,CANDY - CHECKLANE



=== METRIC SUMMARY (Final) ===


,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,TRANS_TIME
mean,1.30,2.89,-0.54,-0.02,"1,562.46"
std,0.93,3.35,1.25,0.22,399.71
min,0.00,0.00,-180.00,-55.93,0.00
25%,1.00,1.26,-0.66,0.00,"1,308.00"
50%,1.00,2.00,0.00,0.00,"1,614.00"
75%,1.00,3.39,0.00,0.00,"1,844.00"
max,144.00,840.00,3.99,0.00,"2,359.00"


Check For Anomaly - Positive Discounts

In [24]:
retail_positive = (trans['RETAIL_DISC'] > 0).sum()
coupon_positive = (trans['COUPON_DISC'] > 0).sum()

print("=== DATA QUALITY CHECK: DISCOUNTS ===")
print(f"Rows with RETAIL_DISC > 0: {retail_positive}")
print(f"Rows with COUPON_DISC > 0: {coupon_positive}")

if retail_positive > 0 or coupon_positive > 0:
    print("\n⚠️ ANOMALY DETECTED: Positive discount values found!")
    print("These should be negative (discounts reduce price)")
    
    # Examine the anomalous rows
    anomalies = trans[(trans['RETAIL_DISC'] > 0) | (trans['COUPON_DISC'] > 0)]
    print(f"\nAnomalous rows: {len(anomalies)}")
    print(anomalies.head(10))

=== DATA QUALITY CHECK: DISCOUNTS ===
Rows with RETAIL_DISC > 0: 36
Rows with COUPON_DISC > 0: 0

⚠️ ANOMALY DETECTED: Positive discount values found!
These should be negative (discounts reduce price)

Anomalous rows: 36
         household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
968972            1306  31624096856  310      990941         0         0.01   
1001608           1306  31735306143  318     7409789         0         0.00   
1023085           1321  31803641901  323     1051069         0         0.00   
1023086           1321  31803641901  323     1055863         0         0.00   
1067337             37  31944616637  334     8090539         0         0.00   
1143255           2241  32173242560  353      995242         0         0.00   
1210504            923  32446036423  369     1047226         0         0.00   
1215551           1535  32478752229  370      999270         1         7.98   
1216489           2491  32478901200  370    13945244         0      

# Cell 5: Profile Demographics

In [5]:
demog = tables['hh_demographic']

print("=== DEMOGRAPHIC COVERAGE ===")
print(f"Households with demographics: {len(demog)}")
print(f"Total households in transactions: {trans['household_key'].nunique()}")
print(f"Coverage: {len(demog) / trans['household_key'].nunique() * 100:.1f}%")

print("\n=== DEMOGRAPHIC DISTRIBUTIONS ===")
for col in ['AGE_DESC', 'INCOME_DESC', 'HH_COMP_DESC']:
    print(f"\n{col}:")
    print(demog[col].value_counts())

=== DEMOGRAPHIC COVERAGE ===
Households with demographics: 801
Total households in transactions: 2500
Coverage: 32.0%

=== DEMOGRAPHIC DISTRIBUTIONS ===

AGE_DESC:
AGE_DESC
45-54    288
35-44    194
25-34    142
65+       72
55-64     59
19-24     46
Name: count, dtype: int64

INCOME_DESC:
INCOME_DESC
50-74K       192
35-49K       172
75-99K        96
25-34K        77
15-24K        74
Under 15K     61
125-149K      38
100-124K      34
150-174K      30
250K+         11
175-199K      11
200-249K       5
Name: count, dtype: int64

HH_COMP_DESC:
HH_COMP_DESC
2 Adults No Kids    255
2 Adults Kids       187
Single Female       144
Single Male          95
Unknown              73
1 Adult Kids         47
Name: count, dtype: int64


# Cell 6: Strategic Data Quality Conclusions

### 1. The "Fuel" Anomaly
* **Discovery:** We identified extreme quantity outliers (up to 89,000 units) with low Sales Value.
* **Root Cause:** These are **Fuel** transactions where `Quantity` represents milliliters or points, not distinct items.
* **Impact:** Including these destroys "Units per Basket" and "Average Unit Price" metrics.
* **Solution:** We built a cleaning pipeline (`scripts/01_clean_transactions.py`) to segregate **Fuel transactions** from the main Grocery dataset using a keyword filter.

### 2. The "Misc" Category Nuance
* **Discovery:** The `COUPON/MISC ITEMS` category contained two types of data:
    1. **Points/Tokens:** Transactions with ~23,000 units and $0.002 unit price (Accounting Data).
    2. **Real Merchandise:** Transactions with normal quantities and prices up to $840 (Valid Revenue).
* **Decision:** We could not simply "Keep" or "Drop" the whole category.
* **Solution:** We implemented a **Row-Level Economic Filter** (`Price < $0.05`). This automatically removed the millions of "Points" rows while preserving the valid High-Ticket merchandise.

### 3. Next Steps
* **Data Pipeline:** We have generated a clean dataset: `data/processed/fact_transactions.csv`.
* **Analysis:** We will use this clean data for the **Churn Threshold Analysis** (Notebook 02) to ensure our frequency calculations are not skewed by non-grocery visits.